In [ ]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
import tensorflow
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')
import sklearn.metrics as metrics

In [ ]:
from sklearn.datasets import load_boston
import xgboost
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score
from xgboost import plot_importance
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.layers import Embedding, Dense, LSTM 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import rhinoMorph

from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
rn = rhinoMorph.startRhino()

filepath:  /Users/Moon/opt/anaconda3/lib/python3.7/site-packages
classpath:  /Users/Moon/opt/anaconda3/lib/python3.7/site-packages/rhinoMorph/lib/rhino.jar
RHINO started!


In [ ]:
df1 = pd.read_excel('./학대별 라벨.xlsx', encoding = 'cp949')
df1
# 1 : 정석학대
# 2 : 신체학대
# 3 : 방임
# 4 : 성학대 

,번호,내용,Label
0,1,\n\t\t\t\t\t\t저는 고등학생 여자입니다7살때부터 가정폭력을 당해왔는데요엄...,1
1,2,\n\t\t\t\t\t\t20대 성인 여성입니다. 졸업을 목전에 두고 있는데 아빠의...,1
2,3,\n\t\t\t\t\t\t옛날부터 엄마가 아빠라는 사람한테 가정폭력을 당했습니다 여...,1
3,4,\n1년이라는 짧지 않은 시간 동안 결혼생활이 지옥같았고 그걸 마무리하려고 합니다남...,1
4,5,\n\t\t\t\t\t\t엄마가 돌돌이? 찍찍이로 방 청소 언제 할거냐 여쭤보셔서 ...,1
...,...,...,...
12325,12326,\n이제 예비고1인 학생입니다.요즘 살기가 싫어지네요사건의 전개는 이렇습니다아버지가...,1
12326,12327,\n\t\t\t\t\t\t부모님이 성적을 만족하시지못해요 절대만족하시는법이없죠 힘들...,1
12327,12328,\n저는 고1되는 여자입니다.저는 정말 살기가 싫어요..그렇다고 자살하는 건 바라지...,1
12328,12329,\n저진짜저희가족이해할수가없습니다저희아빠는저를못믿으시구요 저희엄마는 저랑 제동생을차...,1


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
df1['Label'].value_counts()

2    5378
1    5267
4     975
3     710
Name: Label, dtype: int64

In [ ]:
labels = df1[['번호','Label']]
labels

,번호,Label
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
12325,12326,1
12326,12327,1
12327,12328,1
12328,12329,1


In [ ]:
classes = sorted(labels.Label.unique())
classes

[1, 2, 3, 4]

In [ ]:
df1.drop(['번호','Label'], axis = 1, inplace = True)
df1

,내용
0,\n\t\t\t\t\t\t저는 고등학생 여자입니다7살때부터 가정폭력을 당해왔는데요엄...
1,\n\t\t\t\t\t\t20대 성인 여성입니다. 졸업을 목전에 두고 있는데 아빠의...
2,\n\t\t\t\t\t\t옛날부터 엄마가 아빠라는 사람한테 가정폭력을 당했습니다 여...
3,\n1년이라는 짧지 않은 시간 동안 결혼생활이 지옥같았고 그걸 마무리하려고 합니다남...
4,\n\t\t\t\t\t\t엄마가 돌돌이? 찍찍이로 방 청소 언제 할거냐 여쭤보셔서 ...
...,...
12325,\n이제 예비고1인 학생입니다.요즘 살기가 싫어지네요사건의 전개는 이렇습니다아버지가...
12326,\n\t\t\t\t\t\t부모님이 성적을 만족하시지못해요 절대만족하시는법이없죠 힘들...
12327,\n저는 고1되는 여자입니다.저는 정말 살기가 싫어요..그렇다고 자살하는 건 바라지...
12328,\n저진짜저희가족이해할수가없습니다저희아빠는저를못믿으시구요 저희엄마는 저랑 제동생을차...


In [ ]:
label_to_cat = dict()
for i in range(len(classes)):
    dummy = np.zeros((len(classes),), dtype = 'int8')
    dummy[i] = 1
    label_to_cat[classes[i]] = dummy
    
label_to_cat
#변환이 잘된것을 확인 

{1: array([1, 0, 0, 0], dtype=int8),
 2: array([0, 1, 0, 0], dtype=int8),
 3: array([0, 0, 1, 0], dtype=int8),
 4: array([0, 0, 0, 1], dtype=int8)}

In [ ]:
y = np.array([label_to_cat[label] for label in labels.Label])
y
#y값을 위에서 지정한 것으로 해주고 y로 지정 

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]], dtype=int8)

In [ ]:
labels['Label'].value_counts()

2    5378
1    5267
4     975
3     710
Name: Label, dtype: int64

In [ ]:
def text_cleaning(doc):
    
    doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        
    return doc

In [ ]:
def define_stopwords(path):
    
    SW = set() 
    #집합형태로 만들어줘야 중복을 제외하고 출력해줌
    #불용어들을 추가할려면 SW.add()이렇게 넣어주면 됨
    
    with open(path, encoding = 'cp949') as f:
        for word in f:
            SW.add(word)
            
    return SW

#with open을 통해 파일을 열고 해당 파일에 있는 단어들을 SW에 넣어줌 

In [ ]:
def text_tokenizing(doc):
    return [word for word in rhinoMorph.onlyMorph_list(rn,doc, pos = ['NNG', 'NNP','NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'], eomi = False) if word not in SW or len(word) > 1]

In [ ]:
SW = define_stopwords('./stopwords-ko_v1.txt')

### 여기서 부터 LDA

In [ ]:
df1 = pd.read_csv('./df_tokenized.csv')
df1

,Unnamed: 0,내용,Token
0,0,\n\t\t\t\t\t\t저는 고등학생 여자입니다7살때부터 가정폭력을 당해왔는데요엄...,"['고등학생', '여자', '가정', '폭력', '당하', '엄마', '아빠', '..."
1,1,\n\t\t\t\t\t\t20대 성인 여성입니다. 졸업을 목전에 두고 있는데 아빠의...,"['성인', '여성', '졸업', '목전', '아빠', '가정', '폭력', '학업..."
2,2,\n\t\t\t\t\t\t옛날부터 엄마가 아빠라는 사람한테 가정폭력을 당했습니다 여...,"['옛날', '엄마', '아빠', '사람', '가정', '폭력', '당하', '여태..."
3,3,\n1년이라는 짧지 않은 시간 동안 결혼생활이 지옥같았고 그걸 마무리하려고 합니다남...,"['시간', '동안', '결혼', '생활', '지옥', '그것', '마무리', '남..."
4,4,\n\t\t\t\t\t\t엄마가 돌돌이? 찍찍이로 방 청소 언제 할거냐 여쭤보셔서 ...,"['엄마', '돌돌이', '찍찍이', '청소', '언제', '여쭈', '이따', '..."
...,...,...,...
12325,12325,\n이제 예비고1인 학생입니다.요즘 살기가 싫어지네요사건의 전개는 이렇습니다아버지가...,"['이제', '예비', '고이', '학생', '요즘', '사건', '전개', '이렇..."
12326,12326,\n\t\t\t\t\t\t부모님이 성적을 만족하시지못해요 절대만족하시는법이없죠 힘들...,"['부모', '성적', '만족', '시지', '못하', '절대', '족하', '힘들..."
12327,12327,\n저는 고1되는 여자입니다.저는 정말 살기가 싫어요..그렇다고 자살하는 건 바라지...,"['고되', '여자', '정말', '그렇', '자살', '바라지', '요하', '스..."
12328,12328,\n저진짜저희가족이해할수가없습니다저희아빠는저를못믿으시구요 저희엄마는 저랑 제동생을차...,"['저진', '저희', '저희', '엄마', '제동', '차별', '아빠', '엄마..."


In [ ]:
tf_vect = TfidfVectorizer( min_df=2, max_df=20000,  ngram_range = ( 1, 2 ),max_features = 10000)

vectorizer = tf_vect.fit_transform(df1['Token'])

In [ ]:
vectorizer

<12330x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 1113363 stored elements in Compressed Sparse Row format>

In [ ]:
n_topics = 20

lda = LatentDirichletAllocation(n_components = n_topics)
lda.fit(vectorizer)

LatentDirichletAllocation(n_components=20)

In [ ]:
names = tf_vect.get_feature_names()
topics = dict()

for idx, topic in enumerate(lda.components_):
    vocab = []
    for i in topic.argsort()[:-(30-1):-1]:
        vocab.append((names[i], topic[i].round(2)))
    print('주제 %d' % (idx+1))
    print([(names[i], topic[i].round(2)) for i in topic.argsort()[:-(30-1):-1]])

주제 1
[('방학 숙제', 5.7), ('아빠 얼굴', 3.68), ('누나', 3.25), ('어떻', 3.11), ('계속 갑자기', 2.45), ('말리 사람', 2.45), ('미치 어떡하', 2.43), ('누나 맨날', 2.38), ('센티', 2.36), ('양은', 2.36), ('차고 주먹', 2.35), ('맨날 오늘', 2.35), ('작은방', 2.24), ('엄마 안계', 2.22), ('글씨', 2.21), ('고무장갑', 2.19), ('컴터', 2.18), ('달려들', 2.18), ('뒤통수', 2.11), ('고추', 2.1), ('차고', 2.09), ('ㅎㅎ', 2.06), ('주먹 때리', 2.05), ('행동인', 2.01), ('안보', 1.98), ('우습', 1.94), ('나가', 1.94), ('안계', 1.88)]
주제 2
[('알콜', 51.74), ('알콜 중독', 35.71), ('중독', 35.5), ('입원', 29.06), ('병원', 25.04), ('마시', 20.47), ('치료', 15.58), ('아빠 알콜', 14.26), ('아빠', 11.03), ('강제 입원', 10.53), ('엄마 알콜', 9.93), ('중독자', 9.75), ('알콜 중독자', 9.72), ('저희', 9.59), ('저희 아빠', 9.35), ('강제', 9.1), ('엄마', 7.2), ('병원 입원', 7.15), ('취하', 5.64), ('중독 치료', 5.33), ('소주', 5.27), ('드리', 4.83), ('아버지', 4.27), ('부탁', 4.15), ('알콜 병원', 4.05), ('안마', 3.92), ('집안', 3.78), ('아버지 알콜', 3.7)]
주제 3
[('ㅋㅋ', 56.21), ('ㅋㅋ ㅋㅋ', 47.99), ('고양이', 7.59), ('남친', 5.58), ('때리', 4.91), ('주민', 4.27), ('그래서', 4.23), ('복도', 4.22), ('선

In [ ]:
import pyLDAvis.sklearn

visual = pyLDAvis.sklearn.prepare(lda_model = lda, dtm = vectorizer, vectorizer = tf_vect)
pyLDAvis.save_html(visual, 'LDA_Visualization.html')
pyLDAvis.display(visual)

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
